In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import h5py
import scipy.io as sio
import skimage.io

from Preprocess_Utilities import *

In [3]:
# data path
route_path = "../Data/090085/"
road_type = np.array(["motorway", "trunk", "primary", "secondary", "tertiary", "unclassified", "track",
                      "residential", "service", "road", "living_street",
                      "motor_trunk_pri_sec_tert_uncl_track", "motor_trunk_pri_sec_tert_uncl"])

path_raw_image = route_path + "090085_20170531.h5"
path_road_mask = np.char.add(np.char.add(np.char.add(route_path+"Road_Data/",road_type),
                                         np.char.add('/', road_type)), '.tif')

In [4]:
# read in raw image
raw_image = np.array(h5py.File(path_raw_image)['scene'])

# read in road mask
cur_path = path_road_mask[11]
print(cur_path)
road_img = skimage.io.imread(cur_path)

# assert 0-1 coding
assert (np.logical_or(road_img == 1, road_img == 0)).all()

../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/motor_trunk_pri_sec_tert_uncl_track.tif


In [5]:
print(type(raw_image), raw_image.shape)
print(type(raw_image), road_img.shape)

<class 'numpy.ndarray'> (7, 7961, 8091)
<class 'numpy.ndarray'> (7961, 8091)


In [6]:
# for type_name, path in zip(road_type, path_road_mask):
#     # read in road mask
#     road_img = skimage.io.imread(path)
#     assert (np.logical_or(road_img == 1, road_img == 0)).all()
    
#     # save road against raw img
#     print(type_name, ": ", path)
#     save_path = "./Result/" + type_name
#     show_image_against_road(1160-500,3100-400, road=road_img, light=2.5, size=1000, figsize=(50,50),
#                            show_raw=False)

# show_image_against_road(1450,1850, light=6, size=200)
# show_image_against_road(1400,2550, light=5, size=250)
# show_image_against_road(1450,2800, light=3, size=150)
# show_image_against_road(1250,3450, light=3, size=150, figsize=(20,20))

In [7]:
image_patch = []
road_patch = []
road_existence = []

for row_offset in [0, 7, 14, 21]:
    for col_offset in [0, 7, 14, 21]:
        cur_img_pch, cur_rd_pch, cur_rd_ex = create_labelled_patches(raw_image, road_img,
                                                                     row_offset=row_offset, = []
                                                                     column_offset=col_offset)
        image_patch.extend(cur_img_pch)
        road_patch.extend(cur_rd_pch)
        road_existence.extend(cur_rd_ex)

image_patch = np.array(image_patch)
road_patch = np.array(road_patch)
road_existence = np.array(road_existence)

print(image_patch.shape, road_patch.shape, road_existence.shape)
# (21845, 7, 28, 28) (21845, 28, 28)

(341593, 7, 28, 28) (341593, 28, 28) (341593,)


In [8]:
# save the file
h5_path = '../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/patch_set.h5'
h5f = h5py.File(h5_path, 'w')
h5f.create_dataset(name='image_patch', shape=image_patch.shape, data=image_patch)
h5f.create_dataset(name='road_patch', shape=road_patch.shape, data=road_patch)
h5f.create_dataset(name='road_existence', shape=road_existence.shape, data=road_existence)
h5f.close()

h5f = h5py.File(h5_path, 'r')
h5_image_patch = np.array(h5f['image_patch'])
h5_road_patch = np.array(h5f['road_patch'])
h5_road_existence = np.array(h5f['road_existence'])
h5f.close()

In [10]:
print((h5_image_patch == image_patch).all(), 
      (h5_road_patch == road_patch).all(), 
      (h5_road_existence == road_existence).all())

True True True


In [ ]:
# [0-21450]
index = 1000

has_road = road_existence[index]
mask = road_patch[index]
patch = np.array([image_patch[index][3].T, image_patch[index][2].T, image_patch[index][1].T]).T

print(has_road, patch.max(), patch.min(), patch.shape, mask.shape)
patch = patch * 0.0001 * 5

# plot raw img
plt.imshow(patch)
plt.show()
plt.clf()

# plot road mask
plt.imshow(mask, cmap=plt.get_cmap('gray'))
plt.show()
plt.clf()